# Notebook ETL_steam_games

Este notebook tiene como objetivo realizar las tareas de Extracción, Transformación y Carga (ETL) de los datos del archivo steam_games.json.gz.

## 1. Importamos librerías

In [1]:
# Importamos pandas para la manipulación y análisis de datos tabulares
import pandas as pd

# Importamos numpy para operaciones numéricas eficientes y manipulación de matrices
import numpy as np

# Importamos gzip para trabajar con archivos comprimidos en formato gzip
import gzip

# Importamos json para trabajar con datos en formato JSON
import json

# Importamos ast (Abstract Syntax Trees) para análisis sintáctico y evaluación de expresiones Python
import ast

# Importamos os para interactuar con el sistema operativo y manipular archivos y directorios
import os

# %load_ext autoreload y %autoreload 2 se utilizan para recargar automáticamente los módulos importados
# Esto es útil cuando estamos desarrollando y queremos que los cambios realizados en un módulo se reflejen automáticamente
%load_ext autoreload
%autoreload 2

# Importamos warnings para controlar las advertencias que pueden surgir durante la ejecución del código
import warnings
warnings.filterwarnings("ignore")  # Ignoramos las advertencias para mantener el flujo de trabajo limpio


Ahora continuamos con los pasos del proceso ETL.

## 2. Carga de los conjuntos de datos

En esta fase del proceso ETL, se lleva a cabo la extracción de datos desde un archivo comprimido en formato *gzip*. El archivo **steam_games.json.gz** contiene información relevante sobre los juegos de la plataforma Steam.

Archivo origen: **steam_games.json.gz** (Ubicado en *Data\Raw\steam_games.json.gz*)

In [2]:
# Ruta al archivo gzip comprimido
ruta_games = '../Data/Raw/steam_games.json.gz'

# Lista para almacenar las filas de datos descomprimidos
filas_games = []

# Se lee cada línea del archivo gzip y se descomprime
with gzip.open(ruta_games) as f:
    for line in f.readlines():
        # Se carga la línea descomprimida como un diccionario JSON
        data = json.loads(line)
        # Se agrega el diccionario a la lista de filas
        filas_games.append(data)

# Se convierte la lista de diccionarios en un DataFrame de pandas
df_games = pd.DataFrame(filas_games)


## 3. Exploración del conjunto de datos

En esta sección, se realizará una exploración inicial del dataset de Steam Games. Este dataset contiene información sobre los juegos disponibles en la plataforma Steam, incluyendo detalles como el nombre del juego, el género, las etiquetas, la descripción, entre otros.

El objetivo de esta exploración es comprender la estructura y las características del dataset, identificar posibles patrones y tendencias, y obtener información relevante para el análisis posterior.

In [3]:
# Se muestran los 5 primeros elementos del DataFrame resultante
df_games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dado que los primeros 5 elementos del dataset están vacíos, procedemos a probar con los últimos

In [4]:
# Se muestran los 5 últimos elementos del DataFrame resultante
df_games.tail()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


Durante la exploración inicial del DataFrame **df_games**, se observó que las primeras 5 filas del conjunto de datos están vacías. Se procedió a verificar el número total de filas vacías para determinar el impacto en la integridad del dataset. Además, se notó que las columnas *genres*, *tags* y *specs* contienen datos estructurados en formato *JSON*, como por ejemplo, listas de géneros y etiquetas asociadas a cada juego. Esta observación sugiere la necesidad de realizar un procesamiento adicional para extraer y analizar adecuadamente la información contenida en estas columnas. Este hallazgo destaca la importancia de realizar una limpieza y preparación exhaustiva de los datos antes de llevar a cabo cualquier análisis o modelado posterior.

In [5]:
# Obtener información general del DataFrame
print("\nInformación general del DataFrame:")
df_games.info()


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


Se observa que **app_name**, **url**, **reviews_url** y **early_access** tienen todos los valores no nulos relacionados a algún **id**. Algunas columnas como **genres**, **tags** y **specs** son listas con formajo *JSON* que deberán ser desanidadas para trabajarlas. Se observa que todas las columnas son de tipo *object*, lo cual no es ideal para trabajar los datos; algunos cambios incluyen: **publisher**, **app_name**, **title**, **url**, **reviews_url** y **developer** a tipo *string*; **release_date** a tipo *date*; **price** y **id** a tipo *float64*; y **early_access** a tipo *boolean*.

## 4. Limpieza y Preprocesamiento

Primero eliminamos todas las columnas en las que **todos** los valores estén **vacíos**:

In [6]:
# Utiliza el método dropna() para eliminar las filas con todos los valores nulos
df_games = df_games.dropna(how='all')

Ahora nos aseguramos de que todos los valores vacíos se adapten al formato **NaN** en el Dataframe

In [7]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en df_games
df_games.replace(['', 'null', 'None'], np.nan, inplace=True)

Luego se decidió renombrar algunas columnas para mayor practicidad al momento de trabajar con otros datasets

In [8]:
# Renombrar las columnas dentro del DataFrame df_games
df_games.rename(columns={'app_name': 'item_name', 'id': 'item_id'}, inplace=True)

Se hizo notable analizar y contabilizar los siguientes casos entre las columnas "publisher" y "developer"

In [9]:
# Casos donde "publisher" está vacío y "developer" tiene un valor
casos_publisher_vacio_developer_valor = df_games[(df_games['publisher'].isnull()) & ~(df_games['developer'].isnull())].shape[0]

# Casos donde "publisher" tiene algún valor y "developer" está vacío
casos_publisher_valor_developer_vacio = df_games[~(df_games['publisher'].isnull()) & (df_games['developer'].isnull())].shape[0]

# Casos donde ambas columnas "publisher" y "developer" están vacías
casos_publisher_vacio_developer_vacio = df_games[(df_games['publisher'].isnull()) & (df_games['developer'].isnull())].shape[0]

print("Casos donde 'publisher' está vacío y 'developer' tiene un valor:", casos_publisher_vacio_developer_valor)
print("Casos donde 'publisher' tiene algún valor y 'developer' está vacío:", casos_publisher_valor_developer_vacio)
print("Casos donde ambas columnas 'publisher' y 'developer' están vacías:", casos_publisher_vacio_developer_vacio)


Casos donde 'publisher' está vacío y 'developer' tiene un valor: 4827
Casos donde 'publisher' tiene algún valor y 'developer' está vacío: 65
Casos donde ambas columnas 'publisher' y 'developer' están vacías: 3234


Se observó que en el conjunto de datos hay casos donde la columna "publisher" tiene valores, mientras que la columna "developer" está vacía. Esta discrepancia puede deberse a la ausencia de datos o a la forma en que se registraron los juegos en la base de datos. Para abordar esta inconsistencia y asegurar una mayor coherencia en los datos, se optó por duplicar los valores de la columna "publisher" en la columna "developer" en los casos donde "publisher" tiene algún valor pero "developer" está vacía. Esta decisión se tomó para evitar la pérdida de información relevante y para simplificar el conjunto de datos, eliminando una columna redundante. Este enfoque busca mejorar la integridad y la utilidad de los datos para futuros análisis y aplicaciones. Es fundamental revisar la idoneidad de esta decisión en el contexto específico de los datos y las necesidades del proyecto.

In [10]:
# Filtrar los casos donde 'publisher' tiene algún valor y 'developer' está vacío
casos_publisher_valor_developer_vacio_df = df_games[~(df_games['publisher'].isnull()) & (df_games['developer'].isnull())]

# Duplicar los valores de 'publisher' en la columna 'developer'
df_games.loc[casos_publisher_valor_developer_vacio_df.index, 'developer'] = casos_publisher_valor_developer_vacio_df['publisher']

# Eliminar la columna 'publisher'
#df_games.drop(columns=['publisher'], inplace=True)


Ahora visualizamos los valores duplicados dentro del Dataframe df_games

In [11]:
# Utiliza value_counts() para contar la frecuencia de cada valor en la columna "id"
conteo_valores = df_games['item_id'].value_counts()

# Utiliza la función any() para verificar si algún valor tiene una frecuencia mayor que 1,
# lo que indica que hay valores duplicados
hay_duplicados = (conteo_valores > 1).any()

# Imprime el resultado
if hay_duplicados:
    print("La columna item_id contiene valores duplicados.")
    # Obtén el recuento de valores duplicados
    duplicados_count = conteo_valores[conteo_valores > 1].sum()
    print(f"Número de valores duplicados en la columna item_id: {duplicados_count}")
else:
    print("Todos los valores en la columna item_id son únicos.")


La columna item_id contiene valores duplicados.
Número de valores duplicados en la columna item_id: 2


Se observa que existen valores duplicados, al ser pocos procedemos a mostrarlos

In [12]:
# Identificar filas duplicadas en función de la columna 'id'
filas_duplicadas = df_games[df_games.duplicated(subset=['item_id'], keep=False)]

# Imprimir las filas duplicadas
print("Filas duplicadas en función de la columna 'item_id':")
filas_duplicadas


Filas duplicadas en función de la columna 'item_id':


,publisher,genres,item_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
88384,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Lo anterior nos devuelve 2 pares de filas con valores duplicados:
* La primera fila tiene como "item_id" el valor de nulo (NaN), pero además cuenta con escasos datos.
* La segunda fila y tercera filas hacen referencia al mismo juego pero con url distinta, un dato irrelevante.
* La última fila también tiene un valor nulo en la columna "item_id" pero dentro de su url se encuentra el valor correspondiente que sería "200260".

Se procederá a completar los valores faltantes en la columna 'item_id' de acuerdo con ciertos criterios establecidos. Si un juego tiene un título pero su 'item_id' es NaN, se asignará el valor 200260 a 'item_id'. Asimismo, se eliminarán las filas que carecen tanto de título como de 'item_id'. Además, se eliminará una fila duplicada con 'item_id' igual a 612880, conservando únicamente la primera aparición. Estos cambios contribuirán a la coherencia y la integridad de los datos, asegurando que el DataFrame esté preparado para análisis y aplicaciones posteriores.

Completar los valores faltantes en la columna 'item_id' y eliminar las filas que no tienen información relevante:

In [13]:
# Completar los valores faltantes en la columna 'item_id' y eliminar las filas sin título
df_games.loc[df_games['item_id'].isnull() & df_games['title'].notnull(), 'item_id'] = 200260

# Eliminar las filas sin título y sin 'item_id'
df_games.dropna(subset=['title', 'item_id'], how='all', inplace=True)


Eliminar la fila duplicada con 'item_id' igual a 612880

In [14]:
# Eliminar la tercera fila duplicada con 'item_id' igual a 612880
df_games = df_games.drop_duplicates(subset=['item_id'], keep='first')


Luego se trabajará con la columna "genres" y para ello requerimos una lista con los géneros únicos

In [15]:
# Obtener todos los géneros presentes en la columna 'genres', incluidos los valores nulos
generos_totales = df_games['genres'].explode()

# Obtener los géneros únicos, incluyendo los valores nulos
generos_unicos = generos_totales.unique()

# Convertir los géneros únicos en una lista y agregar el valor nulo
generos_unicos_lista = generos_unicos.tolist()

# Imprimir la lista de géneros únicos
print("Lista de géneros únicos, incluyendo valores nulos:")
print(generos_unicos_lista)


Lista de géneros únicos, incluyendo valores nulos:
['Action', 'Casual', 'Indie', 'Simulation', 'Strategy', 'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing', 'Early Access', 'Massively Multiplayer', 'Animation &amp; Modeling', 'Video Production', 'Utilities', 'Web Publishing', 'Education', 'Software Training', 'Design &amp; Illustration', 'Audio Production', 'Photo Editing', 'Accounting']


Después completamos la columna de "genres" con los valores de "tags" que pertenezcan a alguna de las categorías de nuestra lista anterior

In [16]:
# Convertir la columna 'tags' a listas si es necesario
df_games['tags'] = df_games['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Completar la columna 'genres' con la información de la columna 'tags'
df_games['genres'] = df_games.apply(
    lambda row: [tag for tag in row['tags'] if tag in generos_unicos_lista] if isinstance(row['tags'], list) else row['genres'],
    axis=1
)


Ahora continuamos revisando si alguna fila contiene algún valor vacío para item_name (lo cuál sabemos de la existenci a de una gracias a info())

In [17]:
# Filtrar el DataFrame para obtener solo las filas donde el valor de "item_name" es nulo
fila_vacia_item_name = df_games[df_games['item_name'].isnull()]

# Imprimir la fila seleccionada
fila_vacia_item_name


,publisher,genres,item_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
90890,NaN,"[Action, Indie]",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN


Gracias al valor de la columna "url" y la información del dataset "steam_games" podemos determinar que se trata del juego 'Duet' y actualizaremos su valor en df_games

In [18]:
# Actualizar el valor de "item_name" en la fila seleccionada
df_games.loc[fila_vacia_item_name.index, 'item_name'] = 'Duet'

# Verificar que el valor se haya actualizado correctamente
df_games.loc[fila_vacia_item_name.index]

,publisher,genres,item_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
90890,NaN,"[Action, Indie]",Duet,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN


También tratremos los valores de "release_date" tratando de convertirlos a tipo datetime y creando una columna "year" que extraiga el año de "release_date"

In [19]:
# Convertir la columna 'release_date' a formato de fecha
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')

# Extraer el año y crear la columna 'year'
df_games['year'] = df_games['release_date'].dt.year.astype('Int64')


Continuamos rellenando los valores faltantes de "year" con la mediana

In [20]:
# Calcula la mediana de la columna 'year'
mediana_year = df_games['year'].median()

# Rellena los valores faltantes en 'year' con la mediana
df_games['year'].fillna(mediana_year, inplace=True)


In [21]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publisher     24073 non-null  object        
 1   genres        31995 non-null  object        
 2   item_name     32133 non-null  object        
 3   title         30084 non-null  object        
 4   url           32133 non-null  object        
 5   release_date  29782 non-null  datetime64[ns]
 6   tags          31971 non-null  object        
 7   reviews_url   32132 non-null  object        
 8   specs         31464 non-null  object        
 9   price         30756 non-null  object        
 10  early_access  32133 non-null  bool          
 11  item_id       32133 non-null  object        
 12  developer     28900 non-null  object        
 13  year          32133 non-null  Int64         
dtypes: Int64(1), bool(1), datetime64[ns](1), object(11)
memory usage: 4.5+ MB


Ahora trataremos la columna "price", para ello debemos visualizar primero todos sus valores únicos

In [22]:
unique_prices = df_games['year'].unique()
unique_prices


<IntegerArray>
[2018, 2017, 2016, 1997, 1998, 2006, 2005, 2003, 2007, 2002, 2000, 1995, 1996,
 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992, 1989, 2010, 2011, 2013, 2012,
 2014, 1983, 1984, 2015, 1990, 1988, 1991, 1985, 1982, 1987, 1981, 1986, 2021,
 2019, 1975, 1970, 1980]
Length: 43, dtype: Int64

Se realizaron transformaciones en la columna "price" del conjunto de datos. Se identificaron ciertos valores como "Free To Play", "Free Demo", entre otros, que no representan un costo monetario y, por lo tanto, se convirtieron en "0" para reflejar su carácter gratuito. Asimismo, se ajustaron valores como "Starting at $499.00" y "Starting at $449.00" a "499.0" y "449.0" respectivamente para reflejar su precio base. Además, aquellos elementos marcados como "Third-party" fueron eliminados y convertidos a valores nulos (NaN),

In [23]:
# Reemplazar los valores específicos con "0"
specific_values = ["Free To Play", "Free to Play", "Free", "Free Demo", "Play for Free!", "Install Now",
                   "Play WARMACHINE: Tactics Demo", "Free Mod", "Install Theme", "Play Now", "Free HITMAN™ Holiday Pack",
                   "Play the Demo", "Free to Try", "Free Movie", "Free to Use"]
df_games['price'].replace(specific_values, "0", inplace=True)

# Reemplazar "Starting at $499.00" con "499.0"
df_games['price'].replace("Starting at $499.00", "499.0", inplace=True)

# Reemplazar "Starting at $449.00" con "449.0"
df_games['price'].replace("Starting at $449.00", "449.0", inplace=True)

# Reemplazar "Third-party" con NaN
df_games['price'].replace("Third-party", np.nan, inplace=True)


Este código transforma la columna 'price' del conjunto de datos de juegos, primero convirtiéndola a tipo numérico para facilitar su manipulación y cálculos posteriores. Luego, rellena los valores faltantes (NaN) con la mediana de los precios existentes, asegurando que los datos estén completos y sean representativos para análisis futuros.

In [24]:
# Convertir la columna a tipo numérico (dejando los NaN)
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')

# Rellenar los valores NaN con la mediana
price_median = df_games['price'].median()
df_games['price'].fillna(price_median, inplace=True)


Finalmente eliminamos algunas columnas con información innecesaria para el proyecto

In [25]:
columnas_eliminar = ['publisher', 'title', 'url', 'release_date', 'reviews_url', 'early_access']
df_games.drop(columns=columnas_eliminar, inplace=True)


In [26]:
# Especificar la ruta del archivo de destino
ruta_destino = "../Data/Processed/Clean_data/steam_games_cleaned.csv"

# Exportar el DataFrame como un archivo CSV
df_games.to_csv(ruta_destino, index=False)


## 5. Guardar del conjunto de datos resultante

In [27]:
# Especifica la ruta del archivo de destino
ruta_destino = "../Data/Processed/Clean_data/steam_games_cleaned.csv"

# Exporta el DataFrame como un archivo CSV
df_games.to_csv(ruta_destino, index=False)
